In [131]:
import os
import javalang

PATH = './data/ttt/'
#java_files = []
#files = os.listdir(PATH) #得到文件夹下的所有文件名称

def find_java(path, java_files):
    files = os.listdir(path) #得到文件夹下的所有文件名称
    for file in files:
        if file[0] == '.':  # 隐藏文件或文件夹跳过
            continue
        if os.path.isdir(path+file):
            find_java(path+file+'/', java_files)
        elif file[-5:].lower()=='.java':
            java_files.append(path+file)

#find_java(PATH, java_files)


def get_ast(filename):
    assert not os.path.isdir(filename)
    try:
        with open(filename, 'r') as f:
            code = f.read()
    except:
        return None
    try:
        ast = javalang.parse.parse(code)
        #seq = trans2tokenseq(ast)
        return ast
    except:
        return None
    
def get_functions(ast):
    return list(ast.filter(javalang.tree.MethodDeclaration))

def get_function_name(function_ast):
    return function_ast.name

def find_functions(path, project_dict):
    files = os.listdir(path) #得到文件夹下的所有文件名称
    for file in files:
        if file[0] == '.':  # 隐藏文件或文件夹跳过
            continue
        if os.path.isdir(path+file):
            find_functions(path+file+'/', project_dict)
        elif file[-5:].lower()=='.java':
            file_path = path+file
            ast = get_ast(file_path)
            if ast == None:
                continue
            functions = get_functions(ast)
            
            project_dict['file count'] += 1
            file_profile = {}
            file_profile['file path'] = file_path
            file_profile['function count'] = len(functions)
            file_profile['functions'] = []
            
            for _, node in functions:
                function_name = get_function_name(node)
                file_profile['functions'].append(function_name)
            
            project_dict['files'].append(file_profile)
            
def find_projects(path):
    result = {}
    result['project count'] = 0
    result['projects'] = []
    files = os.listdir(path) #得到文件夹下的所有文件名称
    for file in files:
        if file[0] == '.':  # 隐藏文件或文件夹跳过
            continue
        if os.path.isdir(path+file):
            project_dict = {}
            project_dict['project name'] = file
            project_dict['project path'] = path+file
            project_dict['file count'] = 0
            project_dict['files'] = []
            find_functions(path+file+'/', project_dict)
            result['projects'].append(project_dict)
            result['project count'] += 1
    return result

In [132]:
result = find_projects(PATH)

In [178]:
from utils import get_sequence

def trans_to_sequences(ast):
    sequence = []
    get_sequence(ast, sequence)
    return sequence

def list_to_seq(l):
    seq = l[0]
    for word in l[1:]:
        seq += ' ' + word
    return seq

In [86]:
t = javalang.parse.parse(code)

In [ ]:
for path, node in t.filter(javalang.tree.MethodDeclaration):
    #print(path)
    print(trans_to_sequences(node))

In [28]:
from utils import get_blocks_v1

def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        result.append(tree_to_index(child))
    return result

def trans2seq(r):
    blocks = []
    get_blocks_v1(r, blocks)
    tree = []
    for b in blocks:
        btree = tree_to_index(b)
        tree.append(btree)
    return tree


def tree_to_token(node):
    token = node.token
    #result = [token if token in vocab else 'max_token']
    result = [token]
    children = node.children
    for child in children:
        result.append(tree_to_token(child))
    return result

def trans2tokenseq(r):
    blocks = []
    get_blocks_v1(r, blocks)
    tree = []
    for b in blocks:  # type(b) = ASTNode
        btree = tree_to_token(b)
        tree.append(btree)
    return tree

In [36]:
from utils import get_token, get_children

def visit(ast, l=0):
    for c, child in ast.children():
        print(' '*l, c, child.__class__.__name__, get_token(child))
        visit(child, l+4)
        
def visit2(ast, l=0):
    print(' '*l, ast.__class__.__name__, get_token(ast))
    for c, child in ast.children():
        visit2(child, l+4)
        
def visit3(ast, l=0):
    print(' ' * l, ast.__class__.__name__, get_token(ast))
    children = ast.children()
    for i in range(len(children)):
        visit2(children[i][1], l+4)

In [73]:
def visit_name(ast, l = 0, add_end = False):
    print(' '*l, get_token(ast))
    for child in get_children(ast):
        visit_name(child, l+4)
        if add_end:# and token.lower() == 'compound':
            print(' '*(l+4), 'End')

In [106]:
for path, node in t.filter(javalang.tree.MethodDeclaration):
    #print(path)
    visit_name(node)

TypeError: 'list' object is not callable